In [1]:
import numpy as np
import gdspy
import os
import cv2
import matplotlib.pyplot as plt
from shapely.geometry import Polygon
import shapely
import tqdm
import matplotlib.image
from geneticalgorithm import geneticalgorithm as ga
from slice_utilities import poly_intersection, slice_gds, cost_reward

In [2]:
in_file = '130nm_edit_0001.gds'
top_cell_name = 'top_io'
out_file='via_test.gds'
bounding_box = [(1190, 730), (1190, 775), (1390, 775), (1390, 730)]
slice_gds(in_file,top_cell_name,bounding_box,out_file)

/home/lior/.local/lib/python3.10/site-packages/gdspy/library.py:2291: UserWarning: [GDSPY] GDSII elements of type BOX are imported as polygons.
  self.read_gds(infile, **kwargs)


Layer: 108, Datatype: 0, Polygons: 82
Layer: 66, Datatype: 0, Polygons: 50
Layer: 61, Datatype: 0, Polygons: 1255
Layer: 62, Datatype: 0, Polygons: 1013
Layer: 63, Datatype: 0, Polygons: 617
Layer: 64, Datatype: 0, Polygons: 405
Layer: 65, Datatype: 0, Polygons: 109
Layer: 127, Datatype: 0, Polygons: 255
Layer: 30, Datatype: 0, Polygons: 2341
Layer: 50, Datatype: 0, Polygons: 4493
Layer: 43, Datatype: 0, Polygons: 247
Layer: 40, Datatype: 0, Polygons: 278
Layer: 10, Datatype: 0, Polygons: 682
Layer: 14, Datatype: 0, Polygons: 109
Layer: 74, Datatype: 0, Polygons: 170
Layer: 73, Datatype: 0, Polygons: 198
Layer: 72, Datatype: 0, Polygons: 1302
Layer: 71, Datatype: 0, Polygons: 1172
Layer: 70, Datatype: 0, Polygons: 1354
Layer: 138, Datatype: 0, Polygons: 53
Layer: 135, Datatype: 0, Polygons: 1


In [ ]:
chip_r = gdspy.GdsLibrary(infile = out_file)

Now that we have the reduced gds file we need to set some specific details to construct the needed masks

In [ ]:
chip_r = gdspy.GdsLibrary(infile='bb_slice.gds')
def test_poly(p,l,t):
    targ=[[493.000, 765.940],
    [493.000, 765.660],
    [651.890, 765.660],
    [651.890, 765.940],
    [493.000, 765.940]]
    poly=gdspy.Polygon(p-targ)
    if (poly.area()<0.01):
        return True
    else:
        return False
# changing layer 65 to remove the target polygon
targ=[[493.000, 765.940],
    [493.000, 765.660],
    [651.890, 765.660],
    [651.890, 765.940],
    [493.000, 765.940]]
cell = gdspy.Cell('target')
cell.add(gdspy.Polygon(targ, layer=500, datatype=500))
chip_r.cells['(65, 0)']
chip_r.cells['target']=cell
chip_r.cells['(65, 0)'].remove_polygons(test_poly)
gds_lib = gdspy.GdsLibrary()
gds_lib.write_gds(out_file[:-4]+'_t.gds', chip_r)

In [ ]:
chip_r.cells.keys()

Building the masks

In [ ]:
list_cost_keys = list(chip_r.cells.keys())
list_cost_keys.remove("top")
list_cost_keys.remove("(127, 0)")
list_cost_keys.remove("target")
print(list_cost_keys)
list_reward_keys=list()
list_reward_keys.append("target")

section_bb = Polygon([(493, 682), (493, 857), (788, 857), (788, 682)])  # Replace with your section coordinates
points = section_bb.exterior.xy
min_x, max_x = int(min(points[0])),int(max(points[0]))
min_y, max_y = int(min(points[1])),int(max(points[1]))
w = int(max_x - min_x)
h = int(max_y - min_y)

reselution = 100
mask_cost = np.zeros((w*reselution,h*reselution))
mask_reward = np.zeros((w*reselution,h*reselution))

for t in list_cost_keys:
    polygons = chip_r.cells[t].get_polygons()
    for p in polygons:
        ver=[]
        for i in range(len(p)):
            ver.append([reselution*(p[i,1]-min_y), reselution*(p[i,0]-min_x)])
        ver = np.array(ver,dtype=int)
        cv2.fillPoly(mask_cost, pts=[ver], color=(255,0,0))


for t in list_reward_keys:
    polygons = chip_r.cells[t].get_polygons()
    for p in polygons:
        ver=[]
        for i in range(len(p)):
            ver.append([reselution*(p[i,1]-min_y), reselution*(p[i,0]-min_x)])
        ver = np.array(ver,dtype=int)
        cv2.fillPoly(mask_reward, pts=[ver], color=(255,0,0))

